# Deep Mailing - XGBoost Model - Reduzindo Dimensões

O objetivo desse notebook é demonstrar a utilizacao do XGBoost para a criacao de arvores de decisão para a predição de CUPS em mailings.

Em primeiro lugar, definimos os imports que iremos usar...

In [55]:
import xgboost
import numpy as np
import os
import sys
import logging
import gc
import pickle as pickle
import pandas as pd
import dateutil.parser as parser
import os.path
import math
from sklearn.metrics import accuracy_score,precision_score,recall_score
from datetime import datetime
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams

Abaixo definimos os diretorios e nomes dos arquivos intermediarios.

In [56]:
log_location = "../logs/"
arquivo_df_pickled_norm = "../intermediate/df.norm.corr.pickle"
arquivo_df_pickled_norm_train = "../intermediate/df.norm.train.pickle"
arquivo_df_pickled_norm_test = "../intermediate/df.norm.test.pickle"
arquivo_df_pickled_norm_train_x = "../intermediate/df.norm.train.x.pickle.npy"
arquivo_df_pickled_norm_train_y = "../intermediate/df.norm.train.y.pickle.npy"
arquivo_df_pickled_norm_test_x = "../intermediate/df.norm.test.x.pickle.npy"
arquivo_df_pickled_norm_test_y = "../intermediate/df.norm.test.y.pickle.npy"

Definimos o versampling que é o minimo de rows com a variavel alvo setado no dataset de treinamento

In [57]:
Oversampling = False
RateOversampling = 0.3

Redefinimos o logger que iremos usar

In [58]:
logger = logging.getLogger()
logging.basicConfig(format="%(asctime)-15s %(message)s",
                    level=logging.DEBUG,
                    filename=os.path.join(log_location,'xgboost.reduzindo.log.' + datetime.now().strftime("%Y%m%d%H%M%S.%f") + '.log'))

Criamos uma função para imprimir tanto no log quanto no notebook...

In [59]:
def print_log(msg):
    logging.debug(msg)
    print(msg)    

Carregamos para a memoria o arquivo normalizado e pickled que foi gerado no notebook de "Preparação de Dados".

In [60]:
print_log("Carregando Pickling normalizado:{}".format(arquivo_df_pickled_norm))    
chamadas = pd.read_pickle(arquivo_df_pickled_norm)

Carregando Pickling normalizado:../intermediate/df.norm.corr.pickle


Verificamos as dimensões do dataframe carregado.... E imprimimos uma amostra do dado que precisamos com apenas as colunas relevantes... Como podemos perceber o nosso modelo considera apenas colunas com valores booleanos (0 ou 1)

In [61]:
chamadas.head(10)

,NORM_STATUS_TELEFONE_Hot,NORM_STATUS_INTERNA_Hot,NORM_STATUS_BUREAU_Bom,NORM_STATUS_INTERNA_Indefinido,NORM_CARTEIRA_A01,NORM_STATUS_TELEFONE_Indefinido,NORM_STATUS_BUREAU_Indefinido,NORM_CARTEIRA_W02,NORM_STATUS_TELEFONE_Bom,NORM_ORIGEM_BUREAU,...,NORM_ORIGEM_BASE_INTERNA,NORM_CARTEIRA_A03,NORM_CARTEIRA_A02,NORM_STATUS_INTERNA_Validado,NORM_CARTEIRA_W01,NORM_STATUS_TELEFONE_Sem_Tel,NORM_STATUS_TELEFONE_Validado,NORM_TENTATIVAS,NORM_LIGACOES,NORM_CUP
0,0,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,1,1,0
2,0,0,0,1,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,1,1,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
6,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
7,0,0,1,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,1,0,0
8,1,1,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0


In [62]:
print_log(chamadas.shape)
chamadas.loc[:, :'NORM_LIGACOES'].head(10)

(9586480, 22)


,NORM_STATUS_TELEFONE_Hot,NORM_STATUS_INTERNA_Hot,NORM_STATUS_BUREAU_Bom,NORM_STATUS_INTERNA_Indefinido,NORM_CARTEIRA_A01,NORM_STATUS_TELEFONE_Indefinido,NORM_STATUS_BUREAU_Indefinido,NORM_CARTEIRA_W02,NORM_STATUS_TELEFONE_Bom,NORM_ORIGEM_BUREAU,...,NORM_STATUS_TELEFONE_Bom_menos,NORM_ORIGEM_BASE_INTERNA,NORM_CARTEIRA_A03,NORM_CARTEIRA_A02,NORM_STATUS_INTERNA_Validado,NORM_CARTEIRA_W01,NORM_STATUS_TELEFONE_Sem_Tel,NORM_STATUS_TELEFONE_Validado,NORM_TENTATIVAS,NORM_LIGACOES
0,0,0,0,1,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1,...,1,0,1,0,1,0,0,0,1,1
2,0,0,0,1,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,1,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
6,0,0,0,1,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0
7,0,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,1,0
8,1,1,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1


Para tratarmos as colunas de forma correta, precisamos eliminar os caracteres especiais das colunas, principalmente o espaco e o sinal de - para isso iremos rodar o codigo abaixo para tratar.

In [63]:
cols = chamadas.columns
cols = cols.map(lambda x: x.replace(' ', '_').replace('-','menos'))
chamadas.columns = cols

Podemos agora analisar o tamanho final do nosso dataframe:

In [64]:
total_chamadas = len(chamadas.index)
print_log("Total chamadas:{}".format(total_chamadas))

Total chamadas:9586480


Perfeito, então vamos embaralhar os dados e gerar os nossos dados de treinamento e teste. Vamos considerar 70% para treinamento e 30 % para teste. No final, apagamos o dataframe lido para economizar memoria

In [65]:
print_log("Criando Pickling de train e teste...")
chamadas = chamadas.sample(int(len(chamadas.index)))
chamadas_train = chamadas.tail(int(len(chamadas.index) * 0.7))
chamadas_test = chamadas.head(int(len(chamadas.index) * 0.3))
del chamadas

Criando Pickling de train e teste...


Criamos uma função para gerar um arquivo de referencia de colunas a serem usadas q que vai ser importante na hora de gerar a arvore de decisao...

In [66]:
def create_column_reference(header_chamadas_x,arquivo_df_pickled_norm_train_x):
    print_log("Criando Arquivo de referencia de colunas...")
    with open(arquivo_df_pickled_norm_train_x+".txt","w") as f:
        counter = 0
        lista_header = list(header_chamadas_x.columns.values)
        for header in lista_header:
            f.write("{}-{}\n".format(counter,header))
            counter=counter+1

Criamos agora os nossos dataframes de X que são as features e as Y que são os alvos de predição. Também removemos qualquer linha em tentativas seja igual a zero, além de criar um arquivo de referencia com as colunas X que serão usadas no modelo. Esses dataframes serão convertidos para matrizes no formato numpy

In [67]:
print_log("Separando colunas em X e Y...")        
create_column_reference(chamadas_train.loc[:, :'NORM_LIGACOES'].head(1), arquivo_df_pickled_norm_train_x)
chamadas_train_x = chamadas_train.loc[:, :'NORM_LIGACOES'].as_matrix()
chamadas_train_y = chamadas_train.NORM_CUP.as_matrix()
chamadas_test_x = chamadas_test.loc[:, :'NORM_LIGACOES'].as_matrix()
chamadas_test_y = chamadas_test.NORM_CUP.as_matrix()


Separando colunas em X e Y...
Criando Arquivo de referencia de colunas...


Após a criação das matrizes numpy, gravamos elas em arquivos.

In [68]:
print_log("Criando arquivos finais em formato NUMPY para consumo pelo algoritmo...")        
np.save(arquivo_df_pickled_norm_train_x,chamadas_train_x)
np.save(arquivo_df_pickled_norm_train_y,chamadas_train_y)
np.save(arquivo_df_pickled_norm_test_x,chamadas_test_x)
np.save(arquivo_df_pickled_norm_test_y,chamadas_test_y)


Criando arquivos finais em formato NUMPY para consumo pelo algoritmo...


Apagamos todos os dados intermediários e rodamos o garbage collector para economizar memória.

In [69]:
print_log("Removendo objetos desnecessarios")        
del chamadas_train_x
del chamadas_train_y
del chamadas_train
del chamadas_test
del chamadas_test_x
del chamadas_test_y
gc.collect()

Removendo objetos desnecessarios


2867

Carregamos os objetos numpy em memoria

In [70]:
print_log("Carregando objetos numpy")        
train_x = np.load(arquivo_df_pickled_norm_train_x)
train_y = np.load(arquivo_df_pickled_norm_train_y)
test_x = np.load(arquivo_df_pickled_norm_test_x)
test_y = np.load(arquivo_df_pickled_norm_test_y)

Carregando objetos numpy


Contamos quantos CUPS existem em treinamento e teste...

In [71]:
msg1 = "Train - CUPS Detectados {} num universo de {}".format(len([y for y in train_y if y >0]),len(train_y))
msg2 = "Test - CUPS Detectados {} num universo de {}".format(len([y for y in test_y if y >0]),len(test_y))
print_log(msg1)
print_log(msg2)

Train - CUPS Detectados 16542 num universo de 6710536
Test - CUPS Detectados 7054 num universo de 2875944


Configuramos os parametros para o XGBoost, especificando que queremos que ele seja o mais exato possivel, que a medida de erro é erro simples e que queremos apenas uma classificacao binaria com um maximo de 1000 interações

In [72]:
param = {}
param['eta'] = 0.3
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'auc'
param['tree_method'] = 'exact'
param['silent'] = 0
param['max_depth'] = 6
num_round = 100

Após a definicão dos paramêtros de teste, criamos as matrizes no formato do XGBoost e treinamos o modelo.

In [73]:
gc.collect()
print_log("Starting model for params:{}".format(param))
dtrain = xgb.DMatrix(train_x, train_y)
dtest = xgb.DMatrix(test_x, test_y)


Starting model for params:{'objective': 'binary:logistic', 'eta': 0.3, 'silent': 0, 'tree_method': 'exact', 'eval_metric': 'auc', 'max_depth': 6}


In [74]:
train_labels = dtrain.get_label()
ratio = float(np.sum(train_labels == 0)) / np.sum(train_labels == 1) 
param['scale_pos_weight'] = ratio
print_log("ratio:{}".format(ratio))

ratio:404.666545762302


In [75]:
gpu_res = {}
booster = xgb.train(param, dtrain, num_round, evals=[], evals_result=gpu_res)

Após o modelo ser treinado, podemos plotar ele... Para verificar que coluna é cada feature no modelo, por favor ver a lista em anexo no final desse notebook.

In [76]:
%matplotlib inline
rcParams['figure.figsize'] = 180,150
plot_tree(booster, rankdir='LR')
plt.show()

Agora, vamos tentar predizer os dados com o nosso modelo treinado...

In [77]:
test_y_pred = booster.predict(dtest)
test_predictions = np.array([value for value in test_y_pred])

E Finalmente medir a precisão da nossa predição... Tanto no total quanto em CUPs detectados.

In [82]:
accuracy = accuracy_score(test_y, test_predictions.round())
precision = precision_score(test_y, test_predictions.round())
recall = recall_score(test_y, test_predictions.round())

print_log("Chamados Total:{}".format(len(test_predictions)))
print_log("CUPS Previstos:{}".format(len([x for x in test_predictions if x > 0.5])))
print_log("CUPS na Base Teste:{}".format(len([x for x in test_y if x > 0.5])))
print_log("Accuracy Total:{}".format(accuracy))
print_log("Precision:{}".format(precision))
print_log("Recall:{}".format(recall))

Chamados Total:2875944
CUPS Previstos:493323
CUPS na Base Teste:7054
Accuracy Total:0.8308892662722223
Precision:0.014213811235235333
Recall:0.9940459313864474


Após, vamos salvar o modelo gerado em um arquivo para reuso...

In [79]:
save_file = "../output/{}.model".format(datetime.now().strftime("%Y%m%d.%H%M%S"))
with open(save_file, 'wb') as fp:
    pickle.dump(booster, fp)    
print_log("Model saved as {}".format(save_file))

Model saved as ../output/20171221.195150.model


In [80]:
%%bash 

cat ../intermediate/df.norm.train.x.pickle.npy.txt  

0-NORM_STATUS_TELEFONE_Hot
1-NORM_STATUS_INTERNA_Hot
2-NORM_STATUS_BUREAU_Bom
3-NORM_STATUS_INTERNA_Indefinido
4-NORM_CARTEIRA_A01
5-NORM_STATUS_TELEFONE_Indefinido
6-NORM_STATUS_BUREAU_Indefinido
7-NORM_CARTEIRA_W02
8-NORM_STATUS_TELEFONE_Bom
9-NORM_ORIGEM_BUREAU
10-NORM_STATUS_INTERNA_Bom
11-NORM_STATUS_TELEFONE_Bom_menos
12-NORM_ORIGEM_BASE_INTERNA
13-NORM_CARTEIRA_A03
14-NORM_CARTEIRA_A02
15-NORM_STATUS_INTERNA_Validado
16-NORM_CARTEIRA_W01
17-NORM_STATUS_TELEFONE_Sem_Tel
18-NORM_STATUS_TELEFONE_Validado
19-NORM_TENTATIVAS
20-NORM_LIGACOES
